In [8]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import harmonypy
import pymn
import scrublet as scr
import gseapy as gp
from gseapy.plot import barplot, dotplot
import bottleneck
import pyreadr
from scipy.io import mmread
import scipy
from sklearn import preprocessing
import random as rd
import scib
from matplotlib_venn import venn2,venn2_circles,venn3
from pyscenic.export import export2loom, add_scenic_metadata
from sklearn.decomposition import NMF
from sklearn import metrics
import palettable
from pyscenic.rss import regulon_specificity_scores
import matplotlib.image as mpimg

In [9]:
sc.set_figure_params(dpi = 200, color_map = 'viridis_r' )
sc.settings.verbosity = 2

/public/home/guogjgroup/ggj/anaconda3/lib/python3.9/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


In [10]:
def type_composition(adata_used, clustering_used, type_displayed):
    
    cluster_stat_all = pd.DataFrame()

    for each_tissue in np.unique(adata_used.obs[type_displayed]).tolist():

        each_tissue_stat = pd.DataFrame(adata_used[adata_used.obs[type_displayed] == each_tissue].obs[clustering_used].value_counts())
        each_tissue_stat.columns = [each_tissue]
        cluster_stat_all = pd.concat([each_tissue_stat, cluster_stat_all], axis = 1)

    cluster_stat_all = cluster_stat_all.fillna(0)
    column_list = cluster_stat_all.columns.tolist()
    cluster_stat_all['all_cell'] = cluster_stat_all.sum(axis=1)

    for i in column_list:

        cluster_stat_all[i] = cluster_stat_all[i]/cluster_stat_all['all_cell']

    del cluster_stat_all['all_cell']

    cluster_stat_all['cluster'] = cluster_stat_all.index
    #cluster_stat_all['cluster'] = cluster_stat_all['cluster'].astype(int)
    cluster_stat_all = cluster_stat_all.sort_values('cluster')

    del cluster_stat_all['cluster']

    with plt.rc_context({"figure.figsize": (16, 3), "figure.dpi": (200)}):
        cluster_stat_all.plot(kind = 'bar', stacked = True)
        plt.grid(False)
    #        plt.axis('off')
        plt.legend(bbox_to_anchor=(1.0, 1.0))

# Harmony

In [7]:
pc_select = 30
res = 0.4
neighbor_used = 30

for i in ['6']:

    integration_name = 'all_theta_inner_removal_hvg' + i
    
    adata = sc.read('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/cnv_high_DEG/cnv_high_differential_gene_expression_raw_all_gene.h5ad')
    adata = adata[adata.obs['cnv_cluster_type'] == 'high']
    
    patient_list = np.unique(adata.obs['cancer_type']).tolist()

    alldata = []

    for patient_id in patient_list:

        patient_subset = adata[adata.obs['cancer_type'] == patient_id].copy()

        sc.pp.calculate_qc_metrics(patient_subset, percent_top=None, log1p=False, inplace=True)
        sc.pp.filter_genes(patient_subset, min_cells=1)

        alldata.append(patient_subset)

    adata_subsample_all = alldata[0].copy()

    for other_patient in list(range(1, len(alldata))):

        other_adata = alldata[other_patient].copy()
        adata_subsample_all_tmp = sc.AnnData.concatenate(adata_subsample_all, other_adata, 
                                                         join = "inner", fill_value = 0, index_unique = None)
        adata_subsample_all = adata_subsample_all_tmp.copy()
        del adata_subsample_all_tmp

    adata = adata_subsample_all.copy()
    
    logFC_cutoff = 0.25
    p_cutoff = 0.05
    combined_p_cutoff = 0.05
    pct_cutoff = 0.25

    all_marker = pd.DataFrame()

    for cancer_type in ['BRCA', 'COAD_READ', 'ESCA', 'HCC', 'ICC', 'LUAD', 'STAD']:

        each_marker = pd.read_csv('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/cancer_module/all_cell/conserved_marker/' + cancer_type + '.markers_no_brain.csv', index_col=0)
        each_marker['Tissue'] = cancer_type
        each_marker = each_marker.sort_values('No_avg_log2FC', ascending=False)

        all_marker = pd.concat([all_marker, each_marker], axis=0)

    all_marker = all_marker[all_marker['No_avg_log2FC'] > logFC_cutoff]
    all_marker = all_marker[all_marker['Yes_avg_log2FC'] > logFC_cutoff]
    all_marker = all_marker[all_marker['No_pct.1'] > pct_cutoff]
    all_marker = all_marker[all_marker['Yes_pct.1'] > pct_cutoff]
    all_marker = all_marker[all_marker['minimump_p_val'] < combined_p_cutoff]
    all_marker = all_marker[all_marker['No_p_val_adj'] < p_cutoff]
    all_marker = all_marker[all_marker['Yes_p_val_adj'] < p_cutoff]

    all_marker_stat = []

    for cancer_type in ['BRCA', 'COAD_READ', 'ESCA', 'HCC', 'ICC', 'LUAD', 'STAD']:

        each_marker = all_marker[all_marker['Tissue'] == cancer_type].index.tolist()

        all_marker_stat.extend(each_marker)

    all_marker_stat = np.unique(all_marker_stat).tolist()   

    mt_gene = [s for s in all_marker_stat if s.startswith('MT-')]
    all_marker_stat = list(set(all_marker_stat).difference(set(mt_gene)))

    gene_list = adata.var.index.tolist() 

    adata = adata[:, list(set(gene_list).difference(set(all_marker_stat)))]
    
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)
    
    sc.pp.highly_variable_genes(adata)
    adata = adata[:, adata.var.highly_variable]
    
    '''
    Default: Using normalized DGE as input, then scale and run PCA. 
             ("You can al`so run Harmony on a sparse matrix of library size normalized expression counts. 
             Harmony will scale these counts, run PCA, and finally perform integration."
             Ref: https://github.com/immunogenomics/harmony)


    The size of local neighborhood (in terms of number of neighboring data points) used for manifold approximation. 
    Larger values result in more global views of the manifold, while smaller values result in more local data being 
    preserved. In general values should be in the range 2 to 100. If knn is True, number of nearest neighbors to be 
    searched. If knn is False, a Gaussian kernel width is set to the distance of the n_neighbors neighbor.
    '''

    #https://github.com/immunogenomics/harmony/issues/123
    #https://github.com/immunogenomics/harmony/issues/24
    #https://github.com/immunogenomics/harmony/issues/65

    sc.pp.scale(adata, max_value=10)

    #https://github.com/immunogenomics/harmony/issues/24
    sc.tl.pca(adata, svd_solver='arpack')
    sc.external.pp.harmony_integrate(adata, ['cancer_type'], basis='X_pca', 
                                     adjusted_basis='X_pca_harmony', theta=[int(i)], max_iter_harmony=100)
    del adata.obsm['X_pca']
    adata.obsm['X_pca'] = adata.obsm['X_pca_harmony']

    sc.pp.neighbors(adata, n_pcs=pc_select, n_neighbors=neighbor_used)
    sc.tl.leiden(adata, resolution=res)
    #sc.tl.tsne(adata)
    sc.tl.umap(adata)

    adata.write('harmony_latest/' + integration_name + '.h5ad')

filtered out 7556 genes that are detected in less than 1 cells
filtered out 7866 genes that are detected in less than 1 cells
filtered out 5869 genes that are detected in less than 1 cells
filtered out 5870 genes that are detected in less than 1 cells
filtered out 8284 genes that are detected in less than 1 cells
filtered out 5028 genes that are detected in less than 1 cells
filtered out 10175 genes that are detected in less than 1 cells


/public/home/guogjgroup/ggj/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/public/home/guogjgroup/ggj/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/public/home/guogjgroup/ggj/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the ne

normalizing counts per cell
    finished (0:00:01)
extracting highly variable genes
    finished (0:00:22)


/public/home/guogjgroup/ggj/anaconda3/lib/python3.9/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


... as `zero_center=True`, sparse input is densified and may lead to large memory consumption
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:16)


2023-02-08 16:47:38,560 - harmonypy - INFO - Iteration 1 of 100
2023-02-08 16:49:15,943 - harmonypy - INFO - Iteration 2 of 100
2023-02-08 16:50:51,897 - harmonypy - INFO - Iteration 3 of 100
2023-02-08 16:52:29,286 - harmonypy - INFO - Iteration 4 of 100
2023-02-08 16:54:06,053 - harmonypy - INFO - Iteration 5 of 100
2023-02-08 16:55:44,945 - harmonypy - INFO - Iteration 6 of 100
2023-02-08 16:57:25,556 - harmonypy - INFO - Iteration 7 of 100
2023-02-08 16:59:03,171 - harmonypy - INFO - Iteration 8 of 100
2023-02-08 17:00:44,506 - harmonypy - INFO - Iteration 9 of 100
2023-02-08 17:02:21,562 - harmonypy - INFO - Converged after 9 iterations


computing neighbors
    using 'X_pca' with n_pcs = 30
    finished (0:00:36)
running Leiden clustering
    finished (0:03:54)
computing UMAP
    finished (0:07:46)
